# run

In [1]:
import os,sys
current_dir = os.getcwd()

# muda o diretorio para o projeto --
# isso é necessario para que o python consiga importar os modulos do projeto
# caso contrario, ele nao consegue achar os modulos
project_root = os.path.join(current_dir, '..')
sys.path.append(project_root)
import pandas as pd
from Scripts.Reads import ReadConfigs
from Scripts.Padronizes import PadronizeProdutos
from pandas import to_datetime as pdDatetime
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas import to_numeric as pdNumeric
from collections import defaultdict
import xlsxwriter

# -- 1 - definindo dfs e colunas
base_dir = os.getcwd()
projeto_dir = os.path.abspath(os.path.join(base_dir, '..', '..'))
pasta_dados = os.path.abspath(os.path.join(projeto_dir,'Data', 'BRUTOS-GERAIS', 'CLIENTES'))
df_clientes_masquerade = pd.read_excel(r'C:\Users\MSI\OneDrive\Desktop\faturamento masquerade\Relatorios_faturamento_Masquerade\Data\BRUTOS-GERAIS\CLIENTES\sh-clientes_final.xlsx')
df_clientes = df_clientes_masquerade[['Cd Uf','Nm Cidade','Dt Cadastro']]
df_estados_regiões = pd.read_csv(r'C:\Users\MSI\OneDrive\Desktop\faturamento masquerade\Relatorios_faturamento_Masquerade\Data\BRUTOS-GERAIS\Regioes-estado.csv')
df_novos = ['regiao', 'Nm cidade', 'Quant']

# -- 2 - definindo a região 
df_clientes = df_clientes.merge(df_estados_regiões[['Cd Uf', 'Região']],
                                on='Cd Uf', how='left' )

# -- 3 definindo periodo 

ano, mes = ReadConfigs()

# -- 4 - filtrando pelo peridodo
primeiro_dia = datetime(ano, mes, 1)
ultimo_dia = (primeiro_dia + relativedelta(months=1)) - relativedelta(days=1)   
df_novos_raw = df_clientes[
        (df_clientes['Dt Cadastro'] >= primeiro_dia) & 
        (df_clientes['Dt Cadastro'] <= ultimo_dia)]
df_novos_raw['Ano'] = ano
df_novos_raw['Mês'] = mes
df_novos_raw['Periodo'] = df_novos_raw['Ano'].astype(str) + '-' + df_novos_raw['Mês'].astype(str).str.zfill(2)
df_novos_raw

C:\Users\MSI\AppData\Local\Temp\ipykernel_5060\792155105.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novos_raw['Ano'] = ano
C:\Users\MSI\AppData\Local\Temp\ipykernel_5060\792155105.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novos_raw['Mês'] = mes
C:\Users\MSI\AppData\Local\Temp\ipykernel_5060\792155105.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,Cd Uf,Nm Cidade,Dt Cadastro,Região,Ano,Mês,Periodo
27,SP,BARUERI,2024-02-19,Sudeste,2024,2,2024-02
64,SP,DIADEMA,2024-02-29,Sudeste,2024,2,2024-02
132,SP,BARUERI,2024-02-19,Sudeste,2024,2,2024-02
227,SP,CARAPICUIBA,2024-02-19,Sudeste,2024,2,2024-02
236,SP,BARUERI,2024-02-19,Sudeste,2024,2,2024-02
237,MG,BELO HORIZONTE,2024-02-27,Sudeste,2024,2,2024-02
567,SP,BARUERI,2024-02-19,Sudeste,2024,2,2024-02
763,SP,BARUERI,2024-02-19,Sudeste,2024,2,2024-02
921,SP,JANDIRA,2024-02-19,Sudeste,2024,2,2024-02
962,RS,PORTO ALEGRE,2024-02-09,Sul,2024,2,2024-02


In [2]:
import os,sys
current_dir = os.getcwd()

# muda o diretorio para o projeto --
# isso é necessario para que o python consiga importar os modulos do projeto
# caso contrario, ele nao consegue achar os modulos
project_root = os.path.join(current_dir, '..')
sys.path.append(project_root)
import pandas as pd 
from Scripts.Reads import ReadRaw, ReadConfigs
from Scripts.Padronizes import PadronizeProdutos
os.chdir("..")

ano, mes = ReadConfigs()

# -- 1 - defines as bases de dados e trata certas colunas
df_nfs, _ = ReadRaw(['sh-nfs_final.xlsx'])
df_nfs = df_nfs[['Vl Total Nota', 'Dt. emissão']]  # seleciona só as colunas necessárias

df_pedidos, _ = ReadRaw(['sh-pedidos_final.xlsx'])
df_pedidos = df_pedidos[['Vl venda', 'Dt. emissão']]

# -- 4 - trandorma em mes-ano
df_pedidos['Dt. emissão'] = df_pedidos['Dt. emissão'].dt.to_period('M')
df_nfs['Dt. emissão'] = df_nfs['Dt. emissão'].dt.to_period('M')

# -- 5 - Agrupa por mês e soma os valores
vendas_mensal = df_pedidos.groupby('Dt. emissão')['Vl venda'].sum().reset_index()
faturado_mensal = df_nfs.groupby('Dt. emissão')['Vl Total Nota'].sum().reset_index()

# -- 7 -  mergga os dois DataFrames pela coluna 'Dt. emissão'
df_raw_mes_ano = pd.merge(
    vendas_mensal,
    faturado_mensal,
    on='Dt. emissão',
    how='outer'  # inclui meses que têm apenas vendas ou apenas faturamento
).fillna(0)

# -- 9 - mud o nome ds colunas
df_raw_mes_ano.columns = ['mes-ano', 'venda', 'faturado']

total_mes_faturado = df_raw_mes_ano['faturado'].sum()
total_mes_venda = df_raw_mes_ano['venda'].sum()

In [ ]:
import os
import pandas as pd

def gerar_relatorio_comparativo(df_raw_mes_ano, df_novos_raw):
    base_path = f'resource/fechamento-comparativo/{ano}'
    nome_arquivo = 'relatorio-comparativo.xlsx'
    mes_str = f'{ano}-{mes:02d}'

    # Total de clientes novos por mês
    clientes_por_mes = df_novos_raw.groupby('Periodo').size().reset_index(name='total_clientes')

    # Merge com venda/faturado
    df_raw_mes_ano['mes-ano'] = df_raw_mes_ano['mes-ano'].astype(str)
    clientes_por_mes['Periodo'] = clientes_por_mes['Periodo'].astype(str)

    df_merge = pd.merge(df_raw_mes_ano, clientes_por_mes, how='left', left_on='mes-ano', right_on='Periodo')
    df_merge.drop(columns=['Periodo'], inplace=True)
    df_merge['total_clientes'] = df_merge['total_clientes'].fillna(0).astype(int)

    # Cria colunas em branco
    df_merge['folha'] = ''
    df_merge['contas a pagar'] = ''
    df_merge['% folha'] = ''
    df_merge['ultimo'] = False

    # Marca o mês atual como o último
    df_merge.loc[df_merge['mes-ano'] == mes_str, 'ultimo'] = True

    # Carrega histórico se existir
    mes_anterior = mes - 1
    df_final = None
    if mes_anterior >= 1:
        pasta_ant = os.path.join(base_path, f'{mes_anterior:02d}')
        caminho_ant = os.path.join(pasta_ant, nome_arquivo)
        if os.path.exists(caminho_ant):
            df_ant = pd.read_excel(caminho_ant)
            df_ant['ultimo'] = False  # remove marcação antiga
            # Verifica se o mês atual já está no histórico (evita duplicar)
            if not df_ant['mes-ano'].astype(str).str.contains(mes_str).any():
                nova_linha = df_merge[df_merge['mes-ano'] == mes_str]
                df_final = pd.concat([df_ant, nova_linha], ignore_index=True)
            else:
                df_final = df_ant
    if df_final is None:
        df_final = df_merge[df_merge['mes-ano'] == mes_str].copy()

    # Criar pasta do mês atual
    pasta_atual = os.path.join(base_path, f'{mes:02d}')
    os.makedirs(pasta_atual, exist_ok=True)
    caminho_saida = os.path.join(pasta_atual, nome_arquivo)

    # Gravar arquivo
    with pd.ExcelWriter(caminho_saida, engine='xlsxwriter') as writer:
        df_final.to_excel(writer, index=False, sheet_name='COMPARATIVO')

        worksheet = writer.sheets['COMPARATIVO']
        total_row = len(df_final) + 1
    print(f"✅ Relatório comparativo salvo em: {caminho_saida}")
gerar_relatorio_comparativo(df_raw_mes_ano, df_novos_raw)

✅ Relatório comparativo salvo em: resource/fechamento-comparativo/2024\02\relatorio-comparativo.xlsx
